In [19]:
# import necessary libraries
import pandas as pd
import requests
import os
import dotenv
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base

In [20]:
# Data Ingestion
dotenv.load_dotenv()

API_KEY = os.getenv('API_KEY')

print(API_KEY)

2YG3QMLUCB4WMGTX


In [21]:
BASE_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey={API_KEY}'

stock_market = []
response = requests.get(BASE_URL)
if response.status_code == 200:
    data = response.json()
    stock_market.append(data)
else:
    print('Unable to fetch data')

In [22]:
stock_market

[{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
   '2. Symbol': 'IBM',
   '3. Last Refreshed': '2025-01-31',
   '4. Output Size': 'Compact',
   '5. Time Zone': 'US/Eastern'},
  'Time Series (Daily)': {'2025-01-31': {'1. open': '256.0500',
    '2. high': '257.2350',
    '3. low': '251.8900',
    '4. close': '255.7000',
    '5. volume': '7203519'},
   '2025-01-30': {'1. open': '250.0000',
    '2. high': '261.8000',
    '3. low': '247.0100',
    '4. close': '258.2700',
    '5. volume': '15381890'},
   '2025-01-29': {'1. open': '225.6200',
    '2. high': '229.4700',
    '3. low': '223.7300',
    '4. close': '228.6300',
    '5. volume': '7079804'},
   '2025-01-28': {'1. open': '224.3200',
    '2. high': '225.7700',
    '3. low': '221.7700',
    '4. close': '225.6600',
    '5. volume': '4485429'},
   '2025-01-27': {'1. open': '222.1900',
    '2. high': '224.3000',
    '3. low': '219.8400',
    '4. close': '224.1300',
    '5. volume': '4898355'},
   '202

In [31]:
# Transformation layer
stock_market = data["Time Series (Daily)"]

df = pd.DataFrame.from_dict(stock_market, orient='index')

# Rename columns for clarity
df.rename(columns={
    "1. open": "Open",
    "2. high": "High",
    "3. low": "Low",
    "4. close": "Close",
    "5. volume": "Volume"
}, inplace=True)

# Reset index to have the date as a column
df.reset_index(inplace=True)
df.rename(columns={"index": "Date"}, inplace=True)


,Date,Open,High,Low,Close,Volume
0,2025-01-31,256.0500,257.2350,251.8900,255.7000,7203519
1,2025-01-30,250.0000,261.8000,247.0100,258.2700,15381890
2,2025-01-29,225.6200,229.4700,223.7300,228.6300,7079804
3,2025-01-28,224.3200,225.7700,221.7700,225.6600,4485429
4,2025-01-27,222.1900,224.3000,219.8400,224.1300,4898355


In [15]:
#Convert datatype
df['Date'] = pd.to_datetime(df["Date"])
df['Open'] = pd.to_numeric(df["Open"])
df['High'] = pd.to_numeric(df["High"])
df['Low'] = pd.to_numeric(df["Low"])
df['Close'] = pd.to_numeric(df["Close"])
df['Volume'] = pd.to_numeric(df["Volume"])

NameError: name 'df' is not defined

In [13]:
df.show()

NameError: name 'df' is not defined

In [43]:
#Loading Layer
Base = declarative_base()

class Stock_Price(Base):
    __tablename__ = 'new_stock'

    id = Column(Integer, primary_key=True)
    Date = Column(DateTime)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Volume = Column(Integer)

In [47]:
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PORT = os.getenv("DB_PORT")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")

In [48]:
dotenv.load_dotenv()
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

In [49]:
Session = sessionmaker(bind=engine)
session = Session()
Base.metadata.create_all(engine)

In [50]:
data_to_insert = df.to_dict(orient='records')
session.bulk_insert_mappings(Stock_Price, data_to_insert)
session.commit()
session.close()